# 如何从工具中访问 RunnableConfig
:::info 前提条件
本指南假定您已熟悉以下概念：
- [LangChain 工具](/docs/concepts/tools)- [自定义工具](/docs/how_to/custom_tools)- [LangChain 表达式语言 (LCEL)](/docs/concepts/lcel)- [配置可运行行为](/docs/how_to/configure/)
:::
如果你有一个调用[聊天模型](/docs/concepts/chat_models/)、[检索器](/docs/concepts/retrievers/)或其他[可运行项](/docs/concepts/runnables/)的[工具](/docs/concepts/tools/)，你可能希望访问这些可运行项的内部事件，或用额外属性配置它们。本指南将展示如何正确手动传递参数，以便你能使用`astream_events()`方法实现这一需求。
工具是[可运行对象](/docs/concepts/runnables/)，在接口层面你可以像对待其他任何可运行对象一样对待它们——你可以正常调用它们的`invoke()`、`batch()`和`stream()`方法。但在编写自定义工具时，你可能需要调用其他可运行对象（如聊天模型或检索器）。为了正确追踪和配置这些子调用，你需要手动访问并传入该工具当前的[`RunnableConfig`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.config.RunnableConfig.html)配置对象。本指南将通过示例展示具体操作方法。
:::caution 兼容性
本指南要求 `langchain-core>=0.2.16`。
:::
## 通过参数类型推断
要从自定义工具中访问引用活动配置对象，您需要在工具签名中添加一个类型为 `RunnableConfig` 的参数。当您调用工具时，LangChain 会检查工具签名，寻找类型为 `RunnableConfig` 的参数，如果存在，则会用正确的值填充该参数。
**注意：** 参数的实际名称并不重要，重要的是其类型。
为了说明这一点，定义一个自定义工具，该工具接收两个参数——一个参数类型为字符串，另一个参数类型为`RunnableConfig`：

In [ ]:
%pip install -qU langchain_core

In [2]:
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool


@tool
async def reverse_tool(text: str, special_config_param: RunnableConfig) -> str:
    """A test tool that combines input text with a configurable parameter."""
    return (text + special_config_param["configurable"]["additional_field"])[::-1]

然后，如果我们使用包含 `configurable` 字段的 `config` 调用该工具，可以看到 `additional_field` 被正确传递：

In [3]:
await reverse_tool.ainvoke(
    {"text": "abc"}, config={"configurable": {"additional_field": "123"}}
)

'321cba'

## 后续步骤
您已经了解了如何在工具内配置和流式传输事件。接下来，请查看以下指南以获取更多关于工具使用的信息：
- [在自定义工具中流式传输子运行事件](/docs/how_to/tool_stream_events/)- 将[工具结果传回模型](/docs/how_to/tool_results_pass_to_model)
您还可以查看工具调用的更多具体用途：
- 构建 [工具使用链和智能体](/docs/how_to#tools)- 从模型获取[结构化输出](/docs/how_to/structured_output/)